In [1]:
import os
import math
import sys
from tkinter import ttk
import statsmodels.api as sm
import numpy as np
from functools import reduce
from operator import and_
from tkinter.filedialog import askopenfilename
from dateutil import relativedelta as rd
import yfinance as yf
from statsmodels.stats.outliers_influence import variance_inflation_factor 
import datetime as dt
from pdb import set_trace as keyboard
import xlrd
from xlrd import open_workbook
from sklearn.decomposition import FastICA
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as pdr
from tkinter import StringVar
import matplotlib.pyplot as plt
from sklearn import linear_model
clf = linear_model.LinearRegression()

import matplotlib

yf.pdr_override()

In [2]:
nDays = 30
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=nDays)

In [3]:
fund_data_raw = pd.read_excel('assets/12239939000192.xlsx')

In [4]:
mask = (fund_data_raw['Data'] > startDate) & (fund_data_raw['Data'] <= endDate)
fund_data = fund_data_raw[mask]
fund_data.set_index('Data', inplace=True)
fund_pctChange = pd.DataFrame(fund_data['Variação'])

In [5]:
ibov_data = pd.read_csv('assets/ibov.csv', sep=';')
tickers = ibov_data.iloc[:,0].values[:-2]

In [6]:
tickers_yf = [ticker + '.SA' for ticker in tickers]

startDate_str = startDate.strftime("%Y-%m-%d")
endDate_str   = endDate.strftime("%Y-%m-%d")
stocks_df_raw = pdr.get_data_yahoo(tickers_yf, start=startDate_str, end=endDate_str)['Close']

[*                      3%                       ]  2 of 67 downloaded

Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 357, in history
    quotes = self._parse_quotes(data["chart"]["result"][0])
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 206, in _parse_quotes
    timestamps = data["timestamp"]
KeyError: 'timestamp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 514, in _download_one_threaded
    period, interval, prepost, proxy)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 527, in _download_one


[**********            21%                       ]  14 of 67 downloaded

Exception in thread Thread-21:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 357, in history
    quotes = self._parse_quotes(data["chart"]["result"][0])
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 206, in _parse_quotes
    timestamps = data["timestamp"]
KeyError: 'timestamp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 514, in _download_one_threaded
    period, interval, prepost, proxy)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 527, in _download_one

[**************        30%                       ]  20 of 67 downloaded

...

[**********************75%***********            ]  50 of 67 downloaded

Exception in thread Thread-66:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 357, in history
    quotes = self._parse_quotes(data["chart"]["result"][0])
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 206, in _parse_quotes
    timestamps = data["timestamp"]
KeyError: 'timestamp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 514, in _download_one_threaded
    period, interval, prepost, proxy)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 527, in _download_one

[**********************79%*************          ]  53 of 67 downloaded

Exception in thread Thread-62:
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 357, in history
    quotes = self._parse_quotes(data["chart"]["result"][0])
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 206, in _parse_quotes
    timestamps = data["timestamp"]
KeyError: 'timestamp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 514, in _download_one_threaded
    period, interval, prepost, proxy)
  File "C:\Anaconda3\lib\site-packages\yfinance\__init__.py", line 527, in _download_one

[*********************100%***********************]  60 of 67 downloaded


In [7]:
stocks_df = stocks_df_raw.pct_change()
stocks_df.columns = [i.replace('.SA', '') for i in stocks_df.columns]
stocks_df.head()

,ABEV3,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,BRAP4,BRFS3,BRKM5,BRML3,...,SMLS3,SUZB3,TAEE11,TIMP3,UGPA3,USIM5,VALE3,VIVT4,VVAR11,WEGE3
Date,,,,,,,,,,,,,,,,,,,,,
2019-07-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-07-31,-0.010324,-0.021795,-0.014417,-0.033851,-0.025424,-0.002153,-0.005973,-0.007132,-0.002345,-0.013149,...,0.043122,-0.013509,NaN,0.017485,-0.012903,-0.007937,-0.003999,-0.003240,NaN,-0.007500
2019-08-01,0.014406,0.015644,-0.009549,-0.008679,-0.008696,-0.008320,-0.030677,0.022149,-0.012338,-0.003997,...,-0.010652,-0.000652,NaN,0.004092,0.000000,-0.028571,-0.028308,0.002677,NaN,-0.016373
2019-08-02,0.003428,0.006301,0.000205,-0.002918,0.002339,0.006837,-0.006525,-0.001464,-0.013980,0.007358,...,-0.010254,-0.014356,NaN,0.026080,0.000000,0.000000,-0.011570,0.006865,NaN,0.013231
2019-08-05,-0.016105,-0.038265,-0.018663,-0.020813,-0.018086,-0.025000,-0.045649,-0.012903,-0.014480,-0.024568,...,-0.030044,-0.001986,NaN,-0.027800,-0.042232,-0.032941,-0.038462,-0.021591,NaN,-0.018534


In [8]:
stocks_df.columns

Index(['ABEV3', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BRAP4', 'BRFS3',
       'BRKM5', 'BRML3', 'BTOW3', 'CCRO3', 'CIEL3', 'CMIG4', 'CPFE3', 'CPLE6',
       'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6',
       'EMBR3', 'ENBR3', 'EQTL3', 'ESTC3', 'FIBR3', 'FLRY3', 'GGBR4', 'GOAU4',
       'GOLL4', 'HYPE3', 'IGTA3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'KROT3',
       'LAME4', 'LREN3', 'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NATU3', 'PCAR4',
       'PETR3', 'PETR4', 'QUAL3', 'RADL3', 'RAIL3', 'RENT3', 'SANB11',
       'SAPR11', 'SBSP3', 'SMLS3', 'SUZB3', 'TAEE11', 'TIMP3', 'UGPA3',
       'USIM5', 'VALE3', 'VIVT4', 'VVAR11', 'WEGE3'],
      dtype='object')

In [9]:
data = fund_pctChange.join(stocks_df)
y = data.iloc[:,0]
X_total = data.iloc[:, 1:]

In [10]:
class Results:
    def __init__(self, rsquared_adj, pvalues, selected_stocks):
        self.rsquared_adj = rsquared_adj
        self.pvalues = pvalues
        self.selected_stocks = selected_stocks

In [11]:
columns_with_problem = X_total.columns.to_series()[np.isnan(X_total).any()]
X_total.drop(columns_with_problem, axis=1, inplace=True)

In [12]:
nCols = X_total.columns.size
nCols

60

In [20]:
predef_stocks = ['NATU3', 'MGLU3', 'RENT3', 'EGIE3', 'ECOR3']
stocks_to_sample = [x for x in X_total.columns if not x in predef_stocks]

from pdb import set_trace as keyboard

N = 300
sensitivity = .15
size_samples = np.random.randint(12-len(predef_stocks), 20-len(predef_stocks), size=N)

result_list = []
rsquared_ajd_list = []

X_total_model = X_total.copy()
X_total_model['Bias'] = 1.

useBias = 0

for iteration in range(N):
    
    while True:
        
        selected_stocks = np.random.choice(stocks_to_sample, size=size_samples[iteration]).tolist() + \
                                                                    predef_stocks
        if useBias:
             selected_stocks += ['Bias']

        X = X_total_model[selected_stocks]

        regressor_OLS = sm.OLS(endog=y, exog=X.values).fit()  
        pvalues_dict = dict(zip(selected_stocks, regressor_OLS.pvalues))

        try:
        
            while max(list(pvalues_dict.values())) > sensitivity:
                stocks_bigPvalue = list({ key:value for (key,value) in pvalues_dict.items() if value > sensitivity}.keys())
                if 'Bias' in stocks_bigPvalue:
                    stocks_bigPvalue.remove('Bias')
                stockToRemove = np.random.choice(stocks_bigPvalue)
                selected_stocks.remove(stockToRemove)
                stocks_bigPvalue.remove(stockToRemove)
                regressor_OLS = sm.OLS(endog=y, exog=X.values).fit()  
                pvalues_dict = dict(zip(selected_stocks, regressor_OLS.pvalues))

            result_list.append(Results(regressor_OLS.rsquared_adj, 
                               pvalues_dict, 
                               selected_stocks))
        
        except:
            pass
        
        if len(pvalues_dict) > 0:
            break
                          


    
from operator import attrgetter
best_rsquaredAdj_obj = max(result_list, key=attrgetter('rsquared_adj'))

print('Number of stocks: %s' % len(best_rsquaredAdj_obj.selected_stocks))
print('\nStocks: %s' % best_rsquaredAdj_obj.selected_stocks)
print('\nR2 adj: %s' % best_rsquaredAdj_obj.rsquared_adj)
print('\np-values: %s' % pd.DataFrame(best_rsquaredAdj_obj.pvalues.values(), list(best_rsquaredAdj_obj.pvalues.keys())))

Number of stocks: 17

Stocks: ['CSAN3', 'CPLE6', 'ENBR3', 'CIEL3', 'EQTL3', 'ABEV3', 'USIM5', 'HYPE3', 'LREN3', 'BBSE3', 'ELET3', 'SMLS3', 'CYRE3', 'JBSS3', 'NATU3', 'MGLU3', 'RENT3']

R2 adj: 0.9999264896205942

p-values:               0
CSAN3  0.015371
CPLE6  0.008477
ENBR3  0.009576
CIEL3  0.007023
EQTL3  0.007761
ABEV3  0.009798
USIM5  0.006679
HYPE3  0.008622
LREN3  0.006946
BBSE3  0.008980
ELET3  0.009547
SMLS3  0.008789
CYRE3  0.008466
JBSS3  0.006545
NATU3  0.011059
MGLU3  0.012664
RENT3  0.033886


In [14]:
best_rsquaredAdj_obj.pvalues

{'BBDC3': 0.18538525241078804}

In [15]:
print('\np-values: %s' % best_rsquaredAdj_obj.pvalues)


p-values: {'BBDC3': 0.18538525241078804}


In [16]:
pd.DataFrame(best_rsquaredAdj_obj.pvalues.values(), list(best_rsquaredAdj_obj.pvalues.keys()))

,0
BBDC3,0.185385


In [17]:
list(best_rsquaredAdj_obj.pvalues.keys())

['BBDC3']

In [18]:
selected_stocks.remove('CCRO3')

ValueError: list.remove(x): x not in list